In [1]:
import sys
sys.path.append("../ARCH_package")

import plot, filtering

import dill
import os

In [2]:
# Import non-synonymous trajectories as exported with basic.load
with open('../Exports/LBC_non-synonymous_threshold.dill', 'rb') as infile:
    lbc = dill.load(infile)


# Create path for exporting
path = f'../Results/Threshold/'
if not os.path.exists(path):
    os.makedirs(path)

path = path + "2%_"

fit_mutation_ids = [traj.mutation for part in lbc for traj in part.trajectories if traj.filter is True]
lbc_cohort = [traj for part in lbc for traj in part.trajectories]

# Cohort filter

In [3]:
# Plot non-synonymous cohort
fig = plot.plot_cohort_LiFT(lbc_cohort, filter_type='2%')
fig.show()
fig.update_layout(title='Non-synonymous cohort LiFT')
fig.write_image(path + "non-synonymous.png", scale=10)
fig.write_image(path + "non-synonymous.svg")

In [4]:
# Cohort gradients

# Compute gradients
for part in lbc:
    for i in range(len(part.trajectories)):
        traj = part.trajectories[i]
        data_first = traj.data.iloc[0]
        data_last = traj.data.iloc[-1]
        gradient = (data_last.AF - data_first.AF)/(data_last.age-data_first.age)
        traj.gradient = gradient

# Cohort gradients
fig = plot.gradient_threshold_plot(lbc)
fig.show()
fig.write_image(path + 'gradient_vaf.png', scale=10)
fig.write_image(path + 'gradient_vaf.svg')

In [5]:
fit_cohort = filtering.model_cohort(lbc)

gene_bar = plot.gene_bar(fit_cohort)[0]
gene_bar.show()
gene_bar.write_image(path + "gene_counts.png", scale=10)
gene_bar.write_image(path + "gene_counts.svg")

## Quality checks

In [6]:
fit_mutation_ids = [traj.mutation for part in lbc for traj in part.trajectories if traj.filter is True]


fig, outlier_pvalue_mutation_list = (
    plot.outlier_pvalue_comparison(lbc, fit_mutation_ids))

fig

In [7]:
fig, mdaf_mutation_list = plot.mdaf_comparison(lbc, fit_mutation_ids)
fig